## Quick Guide


The notebook works as a simple script that calls data from CDMNext using CEIC's Python package, extracts all series from the selected insight, gets all timepoints for these series, builds dataframe, and create a grid of interactive plots with the given dataframes


> #### Installation Guide
>
> 1. Install Python 3.9+
> 2. Install the PyCEIC envorionment running the first lines:<br />
> !pip install --extra-index-url https://downloads.ceicdata.com/python ceic_api_client <br />
>!pip install --extra-index-url https://downloads.ceicdata.com/python ceic_api_client --upgrade

---

> #### Usage Guide
>
> **Import libraries** - cell to define libraries we need to set before running the code<br />
> **Insight Information** - cell to define the insight we want to replicate<br />
> **Function Definitions** - to read or change the existing logic used by the main program<br />
> **DataFrames** - select the dataframes that are going to be plotted<br />
> **Code Execution** - run the main program based on parameters and functions<br />
> 
>
> Visit **[CEIC Main Page](https://www.ceicdata.com/en)** for more information on how to use our data and the PyCEIC package.
>
>
>
> **Instructions**
> 1. Run the 'Installing PyCEIC environment' code block to set up the connection with CDMNext
> 2. Run section 'Import Libraries'
> 3. In 'Credentials' section add your log-in credentials or select 'Not a client' to get a sample data
> 4. Then you can run the rest of the code 

---
> #### More info...
> Request a demo of the platform to get access to all the data: https://www.ceicdata.com/en/ceic-request-a-d

## Intalling PyCEIC environment

In [2]:
!pip install --extra-index-url https://downloads.ceicdata.com/python ceic_api_client
!pip install --extra-index-url https://downloads.ceicdata.com/python ceic_api_client --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://downloads.ceicdata.com/python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.7/156.7 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ceic_api_client: filename=ceic_api_client-2.9.2.278-py3-none-any.whl size=438382 sha256=a3f54fd92625da27c241026d6f411900b7896d7f309aebdec9a0270ca258e6c2
  Stored in directory: /root/.cache/pip/wheels/ea/60/97/fe38f2f6ac35c5659f8cd1c7f2720aad869afcdfb5e5c68232
Successfully built ceic_api_client
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://downloads.ceicdata.com/python


### Import Libraries

In [3]:
# Call the PyCEIC functionalities 
from ceic_api_client.pyceic import Ceic 

import matplotlib.pyplot as plot 
import openpyxl
import numpy as np
import pandas as pd
import requests
import math

# Libraries nedded to include interactive widgets
import ipywidgets as widgets
from ipywidgets import DatePicker, interactive, HBox, VBox, Output
from IPython.display import clear_output, display
from datetime import datetime


# Libraries needed to use bokeh python environment
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models.widgets import DatePicker
from bokeh.layouts import gridplot, column, Row
from bokeh.io import output_notebook
from bokeh.models import CustomJS, Label, CustomJSHover
from bokeh.transform import dodge
import bokeh.palettes
from bokeh.models import HoverTool



In [4]:
# Create the widgets to display the log-in box.
user = widgets.Text(value='', placeholder='Nombre de usuario', description='User name:', disabled=False)
password = widgets.Password(value='', placeholder='Contraseña', description='Password:', disabled=False)

login_button = widgets.Button(description='Log In', disabled=False, button_style='', tooltip='Log In', icon='check')
not_client_button = widgets.Button(description="Not a client", disabled=False, button_style='', tooltip='No soy cliente', icon='')

# is_client = True

def on_login_button_clicked(b):
    global is_client
    is_client = True

def on_not_client_button_clicked(b):
    global is_client
    is_client = False

login_button.on_click(on_login_button_clicked)
not_client_button.on_click(on_not_client_button_clicked)

display(widgets.HBox([login_button, not_client_button]), user, password)


Text(value='', description='User name:', placeholder='Nombre de usuario')

Password(description='Password:', placeholder='Contraseña')

### Insight information

Industrial production in Latin America already cooling after the 2021 revival
  <br>
id: ffe35493-6305-4ab8-83fc-bc18ff0f4baf  <br>
Link:https://insights.ceicdata.com/Industrial-production-in-Latin-America-already-cooling-after-the-2021-revival

In [5]:
def get_series_to_df(id_insight, ini_date):
    
    series_dic = {}
    label_dic = {}
    
    #Download the insight series
    result = Ceic.insight_series(id_insight, start_date = ini_date)
    #
    for s in result.data.items:
        column = s.insight_series.name
        time_points_dict = dict((tp._date, tp.value) for tp in s.series_data.time_points)
        series = pd.Series(time_points_dict)
        series_dic[column] = series
        label_dic[column] = column
        
    df = pd.DataFrame(series_dic)
    
    return df, result, series_dic, label_dic

# Import fil from Drive
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params={'id': id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'id': id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)

If you logged with your credentials you will be able to change the 'id' of the insight you want to call and also select the 'insight_date' to specifiy the initial date of your dataframe. If you are not a client the data would be between '2022-01-01' and '2023-01-01' by default.

In [7]:
if is_client == True:

  print('Client Access')

  Ceic.login(user.value,password.value)
  id = 'ffe35493-6305-4ab8-83fc-bc18ff0f4baf'
  insight_date = '2021-01-01' # Change initial date

  data, result, s, l = get_series_to_df(id, insight_date)
  data

else:
  print('Free Access')
  file_id = '1Y8yYPM_MYdvkzhB_-eBstMcMR2z2pmUs'
  destination = '/content/my_excel_file.csv'
  download_file_from_google_drive(file_id, destination)

  data = pd.read_csv('/content/my_excel_file.csv')
  data.iloc[:,0] = pd.to_datetime(data.iloc[:,0])

  first_column_name = data.columns[0]
  data = data.set_index(first_column_name, drop=True)
  data.index.name = None

Free Access


In [8]:
data

,Industrial Production Index: Argentina,Industrial Production Index: Argentina Weighted,Industrial Production Index: Brazil Weighted,Industrial Production Index: Chile Weighted,Industrial Production Index: Colombia Weighted,Industrial Production Index: Mexico Weighted,Industrial Production Index: Peru Weighted,Industrial Production Index: Uruguay Weighted,Industrial Production Index: Latin America Weighted Average,Industrial Production Index: Brazil,Industrial Production Index: Chile,Industrial Production Index: Colombia,Industrial Production Index: Mexico,Industrial Production Index: Peru,Industrial Production Index: Uruguay,Industrial Production Index: Latin America
2023-01-01,6.253442,0.786206,0.105222,0.028809,0.055843,0.806509,0.053596,0.008346,1.844532,0.269398,0.469939,0.796217,2.800173,1.077854,0.575253,1.748897
2022-12-01,-2.699487,-0.295879,-0.143791,-0.088544,0.049423,0.879549,-0.243209,0.016298,0.173846,-0.376290,-1.207549,0.670779,2.984193,-4.649946,1.146046,-0.590322
2022-11-01,0.842822,0.092378,0.312875,-0.368101,0.174637,1.024452,-0.104836,-0.084522,1.046883,0.818770,-5.020079,2.370211,3.475831,-2.004373,-5.943554,-0.780053
2022-10-01,3.315654,0.363415,0.472882,-0.310842,0.243706,0.949543,0.007950,-0.059997,1.666656,1.237495,-4.239197,3.307630,3.221673,0.151988,-4.218937,0.396615
2022-09-01,4.070943,0.446199,-0.373529,-0.104607,0.324159,1.147186,0.022029,0.038723,1.500159,-0.977496,-1.426611,4.399554,3.892250,0.421167,2.722993,1.871829
2022-08-01,7.540325,0.826463,0.670917,-0.370866,0.516185,1.145398,0.028198,0.132339,2.948634,1.755737,-5.057788,7.005781,3.886185,0.539114,9.306036,3.567913
2022-07-01,5.422013,0.594284,-0.136854,-0.372654,0.309339,0.831231,0.123533,-0.014791,1.334088,-0.358137,-5.082168,4.198423,2.820257,2.361836,-1.040082,1.188877
2022-06-01,7.254484,0.795133,-0.230323,-0.113172,0.650224,1.083917,0.286616,0.051368,2.523764,-0.602736,-1.543413,8.824992,3.677589,5.479845,3.612164,3.814703
2022-05-01,11.708497,1.283318,-0.109517,0.116981,2.199880,1.058144,0.091467,0.170235,4.810509,-0.286598,1.595362,29.857277,3.590143,1.748772,11.970859,8.597759
2022-04-01,4.573717,0.501306,-0.036747,-0.257479,0.673796,0.739513,0.169428,0.055581,1.845396,-0.096165,-3.511445,9.144909,2.509069,3.239319,3.908415,2.823974


### Functions

In [9]:
# Allows bokeh to display visuals in Jupyter Notebook
output_notebook()

def create_chart(dtf, chart_type, chart_style):
    source = ColumnDataSource(dtf)
    p = figure(x_axis_type="datetime", title=chart_style['title'], width=450, height=450)
    p.title.text_font_size = str(chart_style['title_params']['fontsize']) + "pt"
    p.title.align = "center"
    p.title.text_font_style = chart_style['title_params']['fontweight']
    p.xaxis.axis_label = chart_style['xlabel']
    p.yaxis.axis_label = chart_style['ylabel']
    p.toolbar.autohide = True
    
    bar_width = 0.9 * 30 * 24 * 60 * 60 * 1000 / len(dtf.columns[:])
    
    colors = bokeh.palettes.d3['Category20'][20]

    for i, value_column in enumerate(dtf.columns[:]):
        if chart_type == 'line':
            TOOLTIPS = [
            ("Value", "$y{0.0}")
                ]

            p.line(x='index', y=value_column, source=source, legend_label=value_column, color = colors[i])          
            p.circle(x='index', y=value_column, source=source, legend_label=value_column, color = colors[i])
            # agregar herramienta de hover
            hover = HoverTool(tooltips=TOOLTIPS)
            p.add_tools(hover)
            
        elif chart_type == 'bar':

            formatter = CustomJSHover(args=dict(source=source),
                                    code="compute percentage with JavaScript based on source, data_x, and data_y")
            TOOLTIPS = [
            ("Value", "@height{0.0}")
                ]

            #  The `{%}` is needed just to trigger the formatter.
            # HoverTool(tooltips=TOOLTIPS, formatters={'@y':formatter})


            p.vbar(x=dodge('index', i * bar_width, range=p.x_range), top=value_column, source=source, width=bar_width, legend_label=value_column, color = colors[i])
                        # agregar herramienta de hover

            hover = p.select(dict(type=HoverTool))
            hover.tooltips = [("Value", "@height")]
            # p.add_tools(hover)
            
  
    p.legend.label_text_font_size = "8pt"
    p.legend.click_policy = "mute"
    
    # Legend glyph (marker) properties
    p.legend.glyph_width = 8
    p.legend.glyph_height = 8
    
    citation = Label(x=300, y=-50, x_units='screen', y_units='screen',
                 text='Source: CEIC Data', render_mode='css',
                 text_font_size='8pt', border_line_alpha=1.0,
                 background_fill_color=None, background_fill_alpha=1.0)
    
    p.add_layout(citation)
    
    # Legend background and border properties
#     p.legend.background_fill_color = "lightgray"
    
    start_picker = DatePicker(title="Start Date", value=pd.to_datetime(dtf.index.min()).strftime('%Y-%m-%d'), width=150)
    end_picker = DatePicker(title="End Date", value=pd.to_datetime(dtf.index.max()).strftime('%Y-%m-%d'), width=150)

    
    callback = CustomJS(args=dict(source=source, start_picker=start_picker, end_picker=end_picker), code="""
        var start_date = new Date(start_picker.value);
        var end_date = new Date(end_picker.value);
        var data = source.data;
        
        var new_data = {'index': []};
        
        for (var key in data) {
            if (key != 'index') {
                new_data[key] = [];
            }
        }
        
        for (var i = 0; i < data['index'].length; i++) {
            var date = new Date(data['index'][i]);
            
            if (date >= start_date && date <= end_date) {
                new_data['index'].push(data['index'][i]);
                for (var key in new_data) {
                    if (key != 'index') {
                        new_data[key].push(data[key][i]);
                    }
                }
            }
        }
        
        source.data = new_data;
        source.change.emit();
    """)
    
    start_picker.js_on_change("value", callback)
    end_picker.js_on_change("value", callback)

    return p, (Row(start_picker, end_picker),)


def plot_dataframes(dataframes, chart_types, chart_styles):
    rows = []
    for i, (df, chart_type, chart_style) in enumerate(zip(dataframes, chart_types, chart_styles)):
        p, widgets = create_chart(df, chart_type, chart_style)
        rows.append(column([column(*widgets), p]))

    grid = gridplot([rows[0:2], rows[2:]])
    show(grid)


# # Keep your existing dtf_list, chart_types, and chart_styles
# plot_dataframes(dtf_list, chart_types, chart_styles)


### Dataframes
> We have to slice the big dataframe into smaller datasets to plot each one of them

In [10]:
dtf1 = data.loc[:,['Industrial Production Index: Latin America Weighted Average'
                   ,'Industrial Production Index: Latin America']].copy()

dtf2 = data.loc[:,['Industrial Production Index: Argentina',
                   'Industrial Production Index: Brazil',
                   'Industrial Production Index: Chile',
                   'Industrial Production Index: Colombia',
                   'Industrial Production Index: Mexico',
                   'Industrial Production Index: Peru',
                   'Industrial Production Index: Uruguay']].copy()

dtf3 = data.loc[:,['Industrial Production Index: Argentina',
                   'Industrial Production Index: Brazil',
                   'Industrial Production Index: Colombia',
                   'Industrial Production Index: Mexico',]].copy()

dtf4 = data.loc[:,['Industrial Production Index: Peru',
                   'Industrial Production Index: Chile',
                   'Industrial Production Index: Uruguay']].copy()

### Styles
> The style of each chart is defined separately 

In [11]:
dtf_list = [dtf1,dtf2,dtf3,dtf4]
chart_types = ['bar','bar','line','line']  
chart_styles = [
    {
        "title": "Industrial Production Index: Latin America",
        "title_params": {
            "fontsize": 10,
            "fontweight": "bold",
            "pad" : 25
        },
        "xlabel": "Month",
        "ylabel": "%"
    },
    {
        "title": "Industrial Production Index: Latin America",
        "title_params": {
            "fontsize": 10,
            "fontweight": "bold",
            "pad" : 25
        },
        "xlabel": "Month",
        "ylabel": "%"
    },
    {
        "title": "Moderate Slowdown",
        "title_params": {
            "fontsize": 10,
            "fontweight": "bold",
            "pad" : 25
        },
        "xlabel": "Month",
        "ylabel": "%",
    },
    {
        "title": "More Pronounced Slowdown",
        "title_params": {
            "fontsize": 10,
            "fontweight": "bold",
            "pad" : 25
        },
        "xlabel": "Month",
        "ylabel": "%",
    }
]

# Code execution

### Industrial production in Latin America already cooling after the 2021 revival

Industrial activity in Latin America is starting to slow down, according to data from CEIC's Latin America Economic Activity Tracker. The industrial production of seven Latin American countries (Argentina, Brazil, Chile, Colombia, Peru, Mexico, and Uruguay) grew at a weighted average of 1% y/y in November 2022, the third deceleration in a row following a recent peak of 5.1% y/y in May 2022.

The positive change figure for the region is driven by the industrial production in Argentina, Brazil, Colombia and Mexico, which grew at positive rates in November 2022. On the other hand, in Chile, Peru and Uruguay industrial production is already lower than 2021. Brazil is a specific case, recording four straight months of positive growth rate as of November after decreases on an annual basis for 11 consecutive months between August 2021 and July 2022. Industrial Production in Brazil was affected by the severe drought observed in H2 2021, that increased energy costs for producers. In addition, rising inflation affected households' purchasing power, further discouraging the production of industrial goods.

Latin America's industrial sector is faced with different challenges stemming from the current macroeconomic environment. Higher consumer prices are negatively impacting household demand for consumer goods, mainly manufactured ones. Further, high input prices and high interest rates are putting the brakes on industrial activity. Finally, Latin America is already starting to feel the lower external demand as a consequence of the global slowdown, decreasing the export revenues from the key trade partners, which could lead to further slowdown in the coming months, until demand for industrial goods strengthens again.

In [12]:
plot_dataframes(dtf_list, chart_types, chart_styles)

*Note that the charts can be updated interactively and downloaded in .png 